# Review of content

## Boolean Mask and Boxplot for outliers

In [7]:
#sns.histplot(data['GrLivArea'], bins=200,kde = True); #for distribution
#sns.boxplot(data=data, x='GrLivArea') #for outliers
# Saving the indexes corresponding to rows 
# without the absurd -1 value
# and without the mansion
boolean_mask = (data['GrLivArea']>0) & (data['GrLivArea']<5000)

# Applying the boolean filteraing
data = data[boolean_mask].reset_index(drop=True)

# Visualize boxplot again
data[['GrLivArea']].boxplot(); 

NameError: name 'data' is not defined

## Scaler

In [25]:


rb_scaler = RobustScaler() 

# Step 1- Fit the scaler to the `GrLiveArea`
# to "learn" the median value and the IQR

rb_scaler.fit(data[['GrLivArea']]) 

# 2-Scale/Transform
# <-> apply the transformation (value - median) / IQR for every house

data['GrLivArea'] = rb_scaler.transform(data[['GrLivArea']]) 

data.head()

NameError: name 'RobustScaler' is not defined

## One Hot Encoder

In [26]:
# Check unique values for streets (3)
print(f"The unique values for 'Street' are {data.Alley.unique()}")

# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse = False) 

# Fit encoder
ohe.fit(data[['Alley']]) 

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "Street" column
data[ohe.get_feature_names_out()] = ohe.transform(data[['Alley']])

# Drop the column "Street" which has been encoded
data.drop(columns = ["Alley"], inplace = True)

# Show the dataset
data.head(3)

NameError: name 'data' is not defined

## Correlation heatmap

In [27]:
# Heatmap of pairwise correlations
correlation_matrix = data.corr()
column_names = correlation_matrix.columns
sns.heatmap(correlation_matrix, xticklabels=column_names, yticklabels=column_names,cmap= "bwr");

NameError: name 'data' is not defined

# Questions to ask

Probably we should undersample the min_revenues column, and as well, find out, if there are so many companies with revenues, why didn't they raise?? how could we answer this question? the 99% remaining raised money

Should a model be created for companies that raised on a that want to raise on b, and for those that raised on a and b, that want to raise on c?

Is less employees bad, average, good, so that we applie an ordinal encoder?

Should remove names and id's?

# Beginning

In [28]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [29]:
path_crunch = '../raw_data/crunchbase.csv'

In [30]:
crunch = pd.read_csv(path_crunch)

/var/folders/fg/0mh3ddwn6ls4kf4146hxrxb00000gn/T/ipykernel_4760/3822556600.py:1: DtypeWarning: Columns (4,11,16,17,18,30,31,32,33,36,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  crunch = pd.read_csv(path_crunch)


In [31]:
crunch.head()

,_id,acquisitions,category,country_code,crunchbase_rank,descriptions.long,descriptions.short,email,employee_range,employees,...,series.series_unknown,series.total,series.undisclosed,socials.facebook,socials.linkedin,socials.twitter,techs,traffic.global_rank,traffic.monthly,website
0,ca17699b-2f49-91f4-5863-7216fc07a64e,"[{""_id"":""a07f601f-d80e-4f67-8996-7f80726b25b4""...",company,gb,309.0,eToro is a social trading investment network t...,eToro is a social trading and investment marke...,support@etoro.com,"[501,1000]","[{""_id"":""f916d557-2c47-27eb-ced9-2a49fd8bb982""...",...,35300000.0,2.227000e+08,NaN,http://www.facebook.com/etoro,https://www.linkedin.com/company/etoro/,http://twitter.com/eToro,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",2549.0,20525524.0,https://www.etoro.com/
1,792dcc41-3650-4496-68ba-481f52ccd99e,[],company,gb,3539.0,GeoSpock is a spatial big data platform compan...,GeoSpock is a spatial big data platform compan...,info@geospock.com,"[11,50]","[{""_id"":""23f56976-1141-4d25-9a0c-7f40e60acebd""...",...,8041980.0,3.824077e+07,NaN,NaN,https://www.linkedin.com/company/geospock-ltd-/,https://twitter.com/geospock,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",6857390.0,709.0,https://www.geospock.com/en/
2,35d239af-cf06-4f04-9387-64a079b641cd,[],company,gb,835.0,Gymshark is an online retailer and technology ...,Gymshark is an online retailer and technology ...,corporate@gymshark.com,"[501,1000]","[{""_id"":""def64484-afd8-482b-82a4-258fe62f7639""...",...,NaN,2.617301e+08,NaN,https://www.facebook.com/Gymshark/,https://www.linkedin.com/company/gymshark/,https://twitter.com/Gymshark,"[{""name"":""IPv6"",""category"":""server""},{""name"":""...",6911.0,7464935.0,https://www.gymshark.com/
3,bb06c586-aac4-4657-39d1-1e428f3f08da,[],company,gb,1450.0,Moneybox now offers a range of saving and inve...,Moneybox is a mobile savings and investment ap...,support@moneyboxapp.com,"[101,250]","[{""_id"":""e9e9034d-eee5-eb7b-836e-db9636a53cce""...",...,NaN,6.986406e+07,NaN,https://www.facebook.com/moneyboxteam/,https://www.linkedin.com/company/digital-moneybox,https://twitter.com/moneyboxteam,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",148195.0,210454.0,https://www.moneyboxapp.com
4,99c2b07b-bbf5-dcac-f768-7b905e940c0e,[],company,gb,1274.0,Oxford Nanopore Technologies aims to disrupt t...,Oxford Nanopore Technologies develops nanopore...,info@nanoporetech.com,"[251,500]","[{""_id"":""f5e4ea8d-70c9-ac60-c80c-f2d871d30a77""...",...,845040507.0,1.142175e+09,NaN,https://www.facebook.com/pages/Oxford-Nanopore...,https://www.linkedin.com/company/oxford-nanopo...,http://twitter.com/nanopore,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",264163.0,95169.0,http://www.nanoporetech.com


### the Column Category has company, investor and school as unique values
HOWEVER, it has values from employee_range column as well. I have to see later if in the values
after having changed the df, if we need these values

In [32]:
crunch.category.unique()

array(['company', 'investor', 'school', nan, '[1,10]', 'true', '[11,50]',
       '[51,100]'], dtype=object)

In [33]:
crunch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311414 entries, 0 to 311413
Data columns (total 52 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   _id                           311394 non-null  object 
 1   acquisitions                  311348 non-null  object 
 2   category                      311369 non-null  object 
 3   country_code                  311413 non-null  object 
 4   crunchbase_rank               311349 non-null  object 
 5   descriptions.long             64060 non-null   object 
 6   descriptions.short            311332 non-null  object 
 7   email                         112326 non-null  object 
 8   employee_range                136292 non-null  object 
 9   employees                     136302 non-null  object 
 10  industries                    134808 non-null  object 
 11  investors                     3433 non-null    object 
 12  legal_name                    16023 non-null

In [34]:
crunch.techs.unique()

array(['[{"name":"Viewport Meta","category":"mobile"},{"name":"IPhone / Mobile Compatible","category":"mobile"},{"name":"ASP.NET","category":"framework"},{"name":"IIS","category":"web_server"},{"name":"GoDaddy","category":"hosting"},{"name":"ASP.NET 4.0","category":"framework"},{"name":"IIS 7","category":"web_server"}]',
       '[{"name":"Viewport Meta","category":"mobile"},{"name":"IPhone / Mobile Compatible","category":"mobile"},{"name":"SPF","category":"mx"},{"name":"SSL by Default","category":"ssl"},{"name":"Google Analytics","category":"analytics"},{"name":"Apple Mobile Web Clips Icon","category":"mobile"},{"name":"Apache","category":"web_server"},{"name":"Content Delivery Network","category":"cdn"},{"name":"Google Tag Manager","category":"widgets"},{"name":"Font Awesome","category":"widgets"}]',
       '[{"name":"IPv6","category":"server"},{"name":"YouTube","category":"media"},{"name":"Google","category":"hosting"},{"name":"ASP.NET","category":"framework"},{"name":"Google Cloud",

# FILTERED ALL DF TO KEEP VALUES THAT WERE SCRAPED

In [35]:
crunch['scraped'].unique()

array([True, nan], dtype=object)

In [36]:
crunch[crunch['scraped']==True]

,_id,acquisitions,category,country_code,crunchbase_rank,descriptions.long,descriptions.short,email,employee_range,employees,...,series.series_unknown,series.total,series.undisclosed,socials.facebook,socials.linkedin,socials.twitter,techs,traffic.global_rank,traffic.monthly,website
0,ca17699b-2f49-91f4-5863-7216fc07a64e,"[{""_id"":""a07f601f-d80e-4f67-8996-7f80726b25b4""...",company,gb,309.0,eToro is a social trading investment network t...,eToro is a social trading and investment marke...,support@etoro.com,"[501,1000]","[{""_id"":""f916d557-2c47-27eb-ced9-2a49fd8bb982""...",...,35300000.0,2.227000e+08,NaN,http://www.facebook.com/etoro,https://www.linkedin.com/company/etoro/,http://twitter.com/eToro,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",2549.0,20525524.0,https://www.etoro.com/
1,792dcc41-3650-4496-68ba-481f52ccd99e,[],company,gb,3539.0,GeoSpock is a spatial big data platform compan...,GeoSpock is a spatial big data platform compan...,info@geospock.com,"[11,50]","[{""_id"":""23f56976-1141-4d25-9a0c-7f40e60acebd""...",...,8041980.0,3.824077e+07,NaN,NaN,https://www.linkedin.com/company/geospock-ltd-/,https://twitter.com/geospock,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",6857390.0,709.0,https://www.geospock.com/en/
2,35d239af-cf06-4f04-9387-64a079b641cd,[],company,gb,835.0,Gymshark is an online retailer and technology ...,Gymshark is an online retailer and technology ...,corporate@gymshark.com,"[501,1000]","[{""_id"":""def64484-afd8-482b-82a4-258fe62f7639""...",...,NaN,2.617301e+08,NaN,https://www.facebook.com/Gymshark/,https://www.linkedin.com/company/gymshark/,https://twitter.com/Gymshark,"[{""name"":""IPv6"",""category"":""server""},{""name"":""...",6911.0,7464935.0,https://www.gymshark.com/
3,bb06c586-aac4-4657-39d1-1e428f3f08da,[],company,gb,1450.0,Moneybox now offers a range of saving and inve...,Moneybox is a mobile savings and investment ap...,support@moneyboxapp.com,"[101,250]","[{""_id"":""e9e9034d-eee5-eb7b-836e-db9636a53cce""...",...,NaN,6.986406e+07,NaN,https://www.facebook.com/moneyboxteam/,https://www.linkedin.com/company/digital-moneybox,https://twitter.com/moneyboxteam,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",148195.0,210454.0,https://www.moneyboxapp.com
4,99c2b07b-bbf5-dcac-f768-7b905e940c0e,[],company,gb,1274.0,Oxford Nanopore Technologies aims to disrupt t...,Oxford Nanopore Technologies develops nanopore...,info@nanoporetech.com,"[251,500]","[{""_id"":""f5e4ea8d-70c9-ac60-c80c-f2d871d30a77""...",...,845040507.0,1.142175e+09,NaN,https://www.facebook.com/pages/Oxford-Nanopore...,https://www.linkedin.com/company/oxford-nanopo...,http://twitter.com/nanopore,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",264163.0,95169.0,http://www.nanoporetech.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311409,a70808ad-93c0-4ad2-b859-699a25c79ed8,[],company,it,987325,NaN,Studio Lania is a service center that offers a...,info@studiolania.it,"[1,10]",[],...,NaN,NaN,NaN,NaN,NaN,NaN,"[{""name"":""Viewport Meta"",""category"":""mobile""},...",NaN,NaN,http://studiolania.it
311410,c3bc2993-9072-419e-99a3-08d51802dfc5,[],company,it,905145,NaN,Puntoweb provides services in the realm of web...,info@puntoweb-arezzo.it,"[1,10]",[],...,NaN,NaN,NaN,https://www.facebook.com/PuntowebSitiInternetA...,https://www.linkedin.com/in/puntoweb-arezzo-80...,https://twitter.com/puntowebarezzo,[],NaN,NaN,http://www.puntoweb-arezzo.it
311411,8c003064-a0f0-4d79-9ddc-46854a957894,[],company,it,1140965,NaN,"Cerme is a distributor of hydraulic, thermal a...",jobs@cerme.it,"[11,50]",[],...,NaN,NaN,NaN,NaN,NaN,NaN,"[{""name"":""SPF"",""category"":""mx""},{""name"":""LetsE...",NaN,NaN,http://www.cerme.it
311412,1bab4fea-ea50-4bc1-a28a-91d6de49f921,[],company,it,1150184,NaN,Studio Anna Fileppo is an art firm that offers...,info@studioannafileppo.it,"[1,10]",[],...,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,http://www.studioannafileppo.it


There are 136311 values that are == True. Two cells below we see that there are 175103 nan values

In [37]:
crunch[crunch['scraped']== True]['scraped'].value_counts()

True    136311
Name: scraped, dtype: int64

In [38]:
crunch['scraped'].value_counts()

True    136311
Name: scraped, dtype: int64

There are 136311 false nan, which means that there are 136311 True! so indeed we can change it

In [39]:
crunch['scraped'].isna().value_counts()

True     175103
False    136311
Name: scraped, dtype: int64

In [40]:
crunch = crunch[crunch['scraped']==True]

# DROPPED COLUMNS I DON'T NEED

In [41]:
crunch = crunch.drop(columns=['acquisitions','employees', 'techs','salesforce_id', 'crunchbase_rank', 'scraped'])

In [42]:
crunch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136311 entries, 0 to 311413
Data columns (total 46 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   _id                           136311 non-null  object 
 1   category                      136266 non-null  object 
 2   country_code                  136311 non-null  object 
 3   descriptions.long             64053 non-null   object 
 4   descriptions.short            136265 non-null  object 
 5   email                         112297 non-null  object 
 6   employee_range                136256 non-null  object 
 7   industries                    134787 non-null  object 
 8   investors                     3417 non-null    object 
 9   legal_name                    16004 non-null   object 
 10  min_revenues                  79957 non-null   float64
 11  name                          136311 non-null  object 
 12  phone                         108920 non-nul

# CLEANED INDUSTRIES & TRANSFORMED IT IN A LIST
But i will see how i will put the transformed industries in a set

In [43]:
crunch.industries

0         [{"_id":"ae5718dc-8602-c733-e62b-28e6167091ad"...
1         [{"_id":"99c5404c-0ff5-8ee5-d334-4b944ddb8cd9"...
2         [{"_id":"275ddcbc-27de-ff4f-4e50-6dc26ebf9570"...
3         [{"_id":"68250c3a-6cf5-86fb-281e-9072ae2d4fe8"...
4         [{"_id":"16ad1d6c-c7ee-b6b5-701d-32c79a19dfae"...
                                ...                        
311409    [{"_id":"ad091ee0-0259-f129-7777-10f65213584d"...
311410    [{"_id":"e362ac36-2e59-c58d-08fe-e5c45f13bc64"...
311411    [{"_id":"9b7bff94-c36c-b8d4-8789-69670454fe84"...
311412    [{"_id":"d29852ab-4bbf-95f5-6087-563ff127de71"...
311413    [{"_id":"9bff86a3-0dc8-1420-9a63-3b86f51d2f36"...
Name: industries, Length: 136311, dtype: object

### In the following i replace all nan values for '[]' , since i can't extract nan's and it will give me an error
However, since i am already extracting values within a '[]' i can replace the nan's by that

In [44]:
crunch['industries'].isna().value_counts()

False    134787
True       1524
Name: industries, dtype: int64

In [52]:
for index, colname in enumerate(crunch):
    print()
    print(f'this column is {colname}')
    print(crunch[crunch['industries'].notna()][colname].value_counts())
    print()
    print(f"and this is the same column {colname} but counting how many nan's")
    print(crunch[crunch['industries'].notna()][colname].isna().value_counts())


this column is _id
ca17699b-2f49-91f4-5863-7216fc07a64e    1
637d51c7-944a-439d-bbc4-1abb570167eb    1
c1b4d6d0-e15d-4f73-960d-8c316182901c    1
631caaca-edaa-4d7b-b29a-7d6f50a8d4a7    1
34acb787-8476-493f-acec-941d736f96ef    1
                                       ..
b6b210bc-58b9-2abb-d8de-7d7efbfe8235    1
6fd9ca2f-345e-4307-9948-618a211d6e6b    1
5c61f428-600b-4f3c-bf88-cf376ad4a703    1
a56c14af-237c-4f01-9128-496065274e68    1
137bcfdd-ff15-4bfa-8da9-92b8d3a8d9cd    1
Name: _id, Length: 134787, dtype: int64

and this is the same column _id but counting how many nan's
False    134787
Name: _id, dtype: int64

this column is category
company     134385
investor       393
school           9
Name: category, dtype: int64

and this is the same column category but counting how many nan's
False    134787
Name: category, dtype: int64

this column is country_code
gb    53570
de    33190
fr    20681
es    18533
it     8813
Name: country_code, dtype: int64

and this is the same column coun

False    134786
True          1
Name: descriptions.short, dtype: int64

this column is email
suport@tradeinn.com               13
general@barcelonatechcity.com      7
Louise.burry@incisivemedia.com     4
rubber@ronfell.com                 4
info@mysite.com                    4
                                  ..
pete@beckleymarketing.co.uk        1
info@regenerus.org.uk              1
info@elftranslations.co.uk         1
sales@harveycomms.co.uk            1
info@idea-web.org                  1
Name: email, Length: 110809, dtype: int64

and this is the same column email but counting how many nan's
False    111191
True      23596
Name: email, dtype: int64

this column is employee_range
[1,10]          66152
[11,50]         47338
[51,100]         7950
[101,250]        7184
[251,500]        3615
[501,1000]       2536
[1001,5000]         2
[5001,10000]        1
Name: employee_range, dtype: int64

and this is the same column employee_range but counting how many nan's
False    134778
True   

True     119379
False     15408
Name: legal_name, dtype: int64

this column is min_revenues
1.000000e+06    37604
0.000000e+00    34186
1.000000e+07     6047
5.000000e+07      941
1.000000e+08      581
5.000000e+08      102
1.000000e+09       81
1.000000e+10       11
Name: min_revenues, dtype: int64

and this is the same column min_revenues but counting how many nan's
False    79553
True     55234
Name: min_revenues, dtype: int64

this column is name
Shift                4
CCS                  4
CDP                  4
IPS                  4
Roth                 3
                    ..
Balanced Up          1
A&q Partnership      1
Go Car Warranty      1
Media Performance    1
Idea Web             1
Name: name, Length: 133950, dtype: int64

and this is the same column name but counting how many nan's
False    134787
Name: name, dtype: int64

this column is phone
15555555555         11
+34 938342661        9
+39 0422 789611      9
+34 932 20 00 89     8
+34 94 458 00 52     5
           

True     132690
False      2097
Name: series.seed, dtype: int64

this column is series.series_a
0.0           61
5000000.0     33
10000000.0    31
7000000.0     21
3000000.0     19
              ..
45019358.0     1
10652273.0     1
6505513.0      1
7957601.0      1
3392057.0      1
Name: series.series_a, Length: 672, dtype: int64

and this is the same column series.series_a but counting how many nan's
True     133737
False      1050
Name: series.series_a, dtype: int64

this column is series.series_b
0.0           23
20000000.0    15
10000000.0     9
25000000.0     9
15000000.0     7
              ..
19898996.0     1
9367236.0      1
90000000.0     1
33882320.0     1
1586689.0      1
Name: series.series_b, Length: 291, dtype: int64

and this is the same column series.series_b but counting how many nan's
True     134357
False       430
Name: series.series_b, dtype: int64

this column is series.series_c
40000000.0    10
0.0            9
25000000.0     6
10000000.0     5
50000000.0     4
 

https://mybaker.co/             3
https://www.chorus.co           2
https://www.sword-apak.com/     2
http://www.bookatable.com       2
https://www.notpla.com/         2
                               ..
http://www.ziggiziggi.com       1
https://assured24.com/          1
http://www.jeffersons.com/      1
https://www.redstones.co.uk/    1
http://www.idea-web.org         1
Name: website, Length: 133743, dtype: int64

and this is the same column website but counting how many nan's
False    133778
True       1009
Name: website, dtype: int64


In [47]:
crunch[crunch['industries'].notna()]['employee_range'].value_counts()

[1,10]          66152
[11,50]         47338
[51,100]         7950
[101,250]        7184
[251,500]        3615
[501,1000]       2536
[1001,5000]         2
[5001,10000]        1
Name: employee_range, dtype: int64

<B>I WILL SEE IF I CAN FILTER AND REMOVE ALL NANS, INSTEAD OF REPLACING LIKE IN THE FOLLOWING CELL

In [6]:
#crunch['industries'].replace(np.nan, '[]', inplace=True)

In [54]:
crunch_where_ind_notna = crunch[crunch['industries'].notna()]

### THE FOLLOWING TWO CELLS SHOULDN'T BE RAN FOR THE MOMENT, since it will transform each cell in a list, and later on we will do the same process but transforming each in a set

<b> THIS BLOCK IS NOT RAN!"!!!!1

In [ ]:
# extract_dict_industries = lambda row: [lines['name'] for lines in json.loads(row)]

In [ ]:
# crunch['industries_list']= crunch['industries'].apply(extract_dict_industries)

In [ ]:
crunch.head()

In [ ]:
crunch = crunch.drop(columns='industries')
crunch.head()


In [ ]:
crunch.columns

In [ ]:
(crunch.scraped_light==True).value_counts()

In [ ]:
crunch.scraped_light.isna().value_counts()

<b> THIS BLOCK IS NOT RAN!"!!!!1

## there are no investors when scraped_light is nan 

In [ ]:
crunch[crunch.scraped_light.isna()]['investors'].isna().value_counts()

### only 3417 values on investors in total


In [ ]:
crunch.investors.isna().value_counts()

In [ ]:
crunch.min_revenues.isna().value_counts()

In [ ]:
crunch[crunch.scraped_light.isna()]['min_revenues'].isna().value_counts()

In [ ]:
crunch.category.unique()

In [ ]:
for column in crunch.columns:
    print()
    print(f'The column name is {column} and the unique values are:' )
    print(crunch[column].unique())
    print()

In [ ]:
crunch.series.isna().value_counts()

## other columns to erase

In [ ]:
#i got to erase series column, 
#scraped_light, and what for is the column category, there is only
#company, investor, school and nan

In [ ]:
crunch['investors'].isna().value_counts()

In [ ]:
#investors probably not important as column, very few values

In [ ]:
crunch['investors'].replace(np.nan, '[]', inplace=True)

In [ ]:
extract_dict_investors = lambda row: [lines['name'] for lines in json.loads(row)]

In [ ]:
crunch['investors_list']= crunch['investors'].apply(extract_dict_investors)

In [ ]:
crunch.head()

### more columns to erase

In [ ]:
crunch = crunch.drop(columns=['investors', 'series'])

In [ ]:
crunch


In [ ]:
crunch[crunch.category == 'school']['series.grant']

In [ ]:
crunch['series.grant'].unique()

In [ ]:
crunch.min_revenues.isna().value_counts()

In [ ]:
crunch[crunch.min_revenues.isna()].loc[:,'series.angel':]

In [ ]:
crunch[crunch.min_revenues.isna()]['series.total'].isna().value_counts()

In [ ]:
crunch[~crunch.min_revenues.isna()]['series.total'].isna().value_counts()

In [ ]:
crunch.columns

In [ ]:
crunch.corr

In [ ]:
test = crunch.dropna()

In [ ]:
test #why th8s?

In [ ]:
#df = crunch.drop_duplicates()

### more columns to drop

In [ ]:
df = crunch.drop(columns= ['_id', 'category', 'country_code', 'employee_range', 'legal_name',
       'min_revenues', 'name',  'series.angel',
       'series.convertible_note', 'series.corporate_round',
       'series.debt_financing', 'series.equity_crowdfunding', 'series.grant',
       'series.initial_coin_offering', 'series.non_equity_assistance',
       'series.post_ipo_debt', 'series.post_ipo_equity', 'series.pre_seed',
       'series.private_equity', 'series.secondary_market', 'series.seed',
       'series.series_a', 'series.series_b', 'series.series_c',
       'series.series_d', 'series.series_e', 'series.series_f',
       'series.series_g', 'series.series_h', 'series.series_unknown',
       'series.total', 'series.undisclosed', 'traffic.global_rank',
       'traffic.monthly',  'industries_list', 'investors_list'])

In [ ]:
df = crunch.drop(columns= [ 'descriptions.long','descriptions.short', 'email','phone', 'scraped_light', 'socials.facebook',
       'socials.linkedin', 'socials.twitter','website',])

In [ ]:
df = df.drop(columns='legal_name')

In [ ]:
df.head()

In [ ]:
for column in df.columns[0:10]:
    print()
    print(f'The column name is {column} and the value is:' )
    print()
    print(f'the first element is of type {type(df[column][0])}')
    print(df[column])
    print()
    

In [ ]:
df.info()

In [ ]:
df['series.angel'].isna().value_counts()

In [ ]:
df['series.angel'].value_counts()

## SUM ALL SERIES

In [ ]:
df.loc[:10, 'series.angel':'series.undisclosed']

In [ ]:
list(enumerate(df))

In [ ]:
total = 0
for (index, colname) in enumerate(df):
    print(index, colname, df[colname][3])
    if index >5 and index <29:
        total = total + df[colname][3]
print(f'total is {total}')

In [ ]:
for (index, colname) in enumerate(df):
    if 'series' in colname:
        df[colname].replace(np.nan, 0.0, inplace=True)

In [ ]:
df.loc[:10, 'series.angel':'series.undisclosed']

In [ ]:
#df['series.angel':'series.undisclosed'].replace(np.nan, 0.0, inplace=True)

In [ ]:
#df['series.angel':'series.undisclosed'].

In [ ]:
df['total funds']= df.iloc[:, 6:31].sum(axis =1)

In [ ]:
(df['total funds']>0).value_counts()

In [ ]:
df.info()

## element & element boolean

In [ ]:
df['series.undisclosed'].value_counts()

In [ ]:
#df[df['series.undisclosed']>0 and df['series.total']>0].value_counts()

In [ ]:
df[df['series.undisclosed']>0].value_counts()

In [ ]:
df['series.undisclosed'].unique()

In [ ]:
rslt_df = df[(df['series.total'] > 0) & 
          (df['series.undisclosed'] > 0)]

In [ ]:
rslt_df.info()

In [ ]:
df['series.undisclosed'].info()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.employee_range.unique()

In [ ]:
df.min_revenues.unique()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['industries_list'] #clustering

In [ ]:
df.category.unique()

In [ ]:
df.head()

# DROPPED MORE COLUMNS - should     have only one function for this

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['category', 'series.angel', 'series.convertible_note',
       'series.corporate_round', 'series.debt_financing',
       'series.equity_crowdfunding', 'series.grant',
       'series.initial_coin_offering', 'series.non_equity_assistance',
       'series.post_ipo_debt', 'series.post_ipo_equity', 'series.pre_seed',
       'series.private_equity', 'series.secondary_market', 'series.seed',
       'series.series_a', 'series.series_b', 'series.series_c',
       'series.series_d', 'series.series_e', 'series.series_f',
       'series.series_g', 'series.series_h', 'series.series_unknown',
       'series.total', 'series.undisclosed'])

In [ ]:
df.info()

In [ ]:
df.employee_range.unique()

In [ ]:
df.employee_range.isna().value_counts() 

In [ ]:
df.min_revenues.unique()

In [ ]:
df.head()

# TREATING EMPLOYEE_RANGE

In [ ]:
#employee_clean = lambda x: x[1:-1].split(',')

In [ ]:
df['employee_range'].unique()

In [ ]:
test = df['employee_range'][0]
test

In [ ]:
employee_clean = lambda x: int(np.mean([ int(a) for a in x[1:-1].split(',')]))

In [ ]:
result = employee_clean(test)

In [ ]:
result

In [ ]:
df['employee_range'].isna().value_counts()

In [ ]:
df.info()

In [ ]:
df = df[~df['employee_range'].isna()]

In [ ]:
df.shape

In [ ]:
#type(result[0])

In [ ]:
df['employee_range']= df['employee_range'].apply(employee_clean)

In [ ]:
df.head()

# TREATING MIN REVENUES

In [ ]:
df.min_revenues.isna().value_counts()

In [ ]:
#moyenne, ou minium categorie car peut etre revenue est faible, on prends choix au momentd e la modelisation

moyenne par industrie

In [ ]:
df['min_revenues'].hist(bins=3)

In [ ]:
df['min_revenues'].unique()

In [ ]:
df['min_revenues'].value_counts()

In [ ]:
something = df[~df['min_revenues'].isna()]['min_revenues']

In [ ]:
something

In [ ]:
type(df['min_revenues'][573])

In [ ]:
#something[0] #WHY?????

# TREATING GLOBAL RANK

In [ ]:
df['traffic.global_rank'].hist()

# FINDING CORRELATION

In [ ]:
df.corr()

In [ ]:
df.head()

# TREATING INDUSTRIES LIST

In [ ]:
type(df['industries_list'][0])

In [ ]:
all_industries = []
for row in df['industries_list']:
    for ele in row:
        if ele not in all_industries:
            all_industries.append(ele)
all_industries

In [ ]:
len(all_industries)

In [ ]:
df.info()

In [ ]:
industries_dict = {
    "Finance & Accounting": [
        "Finance",
        "Financial Services",
        "FinTech",
        "Trading Platform",
        "Asset Management",
        "Banking",
        "Lending",
        "Bitcoin",
        "Cryptocurrency",
        "Financial Exchanges",
        "Angel Investment",
        "Impact Investing",
        "Venture Capital",
        "Accounting",
        "Payments",
        "Consumer Lending",
        "Credit",
        "Personal Finance",
        "Commercial Insurance",
        "Insurance",
        "InsurTech",
        "Auto Insurance",
        "Wealth Management"
    ],
    "Technology & Software": [
        "Artificial Intelligence",
        "Analytics",
        "Marketplace",
        "Information Services",
        "Mobile",
        "SaaS",
        "Smart Cities",
        "E-Commerce",
        "Machine Learning",
        "Predictive Analytics",
        "Natural Language Processing",
        "Market Research",
        "Blockchain",
        "Cyber Security",
        "Industrial Automation",
        "Quantum Computing",
        "Big Data",
        "Privacy",
        "Developer APIs",
        "Apps",
        "B2B",
        "B2C",
        "Computer",
        "Enterprise Software",
        "Database",
        "Computer Vision",
        "Robotics",
        "Transaction Processing",
        "Peer to Peer",
        "Intellectual Property",
        "Android",
        "Cloud Computing",
        "Education",
        "iOS",
        "Hardware",
        "Semiconductor",
        "Collaboration",
        "Communications Infrastructure",
        "IaaS",
        "Productivity Tools",
        "Unified Communications",
        "Virtual Workforce",
        "Identity Management",
        "Security",
        "Internet of Things",
        "E-Learning",
        "Open Source",
        "Operating Systems",
        "Embedded Software",
        "Embedded Systems",
        "Developer Tools"
    ],
    "Health & Wellness": [
        "Biopharma",
        "Biotechnology",
        "Genetics",
        "Health Diagnostics",
        "Nanotechnology",
        "Assistive Technology",
        "Health Care",
        "Information Technology",
        "mHealth",
        "Personal Health"
    ],
    "Marketing & Advertising": [
        "Advertising",
        "Content",
        "Digital Marketing",
        "Digital Media",
        "Social Media Marketing",
        "Content Marketing",
        "Affiliate Marketing",
        "Retail Technology"
    ],
    "Transportation & Logistics": [
        "Logistics",
        "Marine Technology",
        "Automotive",
        "Autonomous Vehicles",
        "Fleet Management",
        "Internet",
        "Rental",
        "Transportation",
        "Marine Transportation",
        "Shipping",
        "Delivery Service",
        "Travel"
    ],
    "Energy & Environment": [
        "Clean Energy",
        "Energy",
        "Industrial Engineering"
    ]
}

In [ ]:
my_test = df['industries_list'][0]
my_test

In [ ]:
df['industries_list'][0]

In [ ]:
for index, row in enumerate(df['industries_list']):
    print(index, row)

## Created a for loop to change industries_list into values from dict. Too long

In [ ]:
industries_dict.keys()

In [ ]:
'Travel' in industries_dict.values()

In [ ]:
industries_dict.values()

In [ ]:
my_test = ['Finance', 'Financial Services', 'FinTech', 'Marketplace', 'Trading Platform']


In [ ]:
set(my_test)

In [ ]:
clean_industries_list = lambda row: set([k for k, v in industries_dict.items() if set(row).intersection(set(v))])

In [ ]:
clean_industries_list(df['industries_list'][0])

In [ ]:
df['industries_list'][50000]

In [ ]:
df['industries_cleaned'] = df['industries_list'].apply(clean_industries_list)

In [ ]:
df.head()

In [ ]:
#do not run this cell
"""
for index, row in enumerate(df['industries_list']):
    my_new_list = []
    for industries_in_cell in row:
        for sector, industries_in_dict in industries_dict.items():
            if industries_in_cell in industries_in_dict:
                if sector not in my_new_list:
                    my_new_list.append(sector)
    df['industries_list'][index]= my_new_list
        
"""



In [ ]:
df['industries_list'][0]

In [ ]:
df

In [ ]:
my_new_list

In [ ]:
industries_dict.values()

In [ ]:
industries_dict.keys()

In [ ]:
for k, v in industries_dict.items():
    print(v)

In [ ]:
df = df.drop(columns='industries_list')

In [ ]:
df

# CHANGED Total funds into 1 or 0 if funds True

In [ ]:
df['total funds']= df['total funds']>0

In [ ]:
df['total funds']= df['total funds'].apply(lambda x: 1 if x == True else 0)

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['name'])

In [ ]:
no_revenues_df = crunch[crunch['min_revenues'].isna()].loc[]

In [ ]:
df[df['total funds']==1]['min_revenues'].value_counts()

In [ ]:
df[df['total funds']==1]['min_revenues'].isna().value_counts()

### 56300 show no info on revenues

In [ ]:
df['min_revenues'].isna().value_counts() 

### from these 56300 companies from which we don't have any info on revenues, 3835 fund raised

In [ ]:
df[df['min_revenues'].isna()]['total funds'].value_counts()

### 79956 companies show info on revenues

In [ ]:
df['min_revenues'].notna().value_counts() 

### from these 79956 companies from which we have revenues, 1 fund raised

In [ ]:
df[df['min_revenues'].notna()]['total funds'].value_counts()

### From 132420 companies, only 3836 raised

In [ ]:
df['total funds'].value_counts()

### And from these 3836 companies that raised, 3835 have no info on revenues!

In [ ]:
crunch.columns

In [ ]:
'series.angel'

In [ ]:
df.head()

1. quel modèles scikitlearn classifier
2. 
- country_code = onehotencoder
- investors list onehotencoder
- industries_cleaned = onehotencoder
- employee_range onehotencoder


### confirming hypothesis on immature startups(Angel, pre-seed, seed, series a) that have no revenues

#### however we know previously that 99% of no revenue companies/startups, were the ones who raised. And only 1% with revenue raised

In [ ]:
crunch

In [ ]:
list(enumerate(crunch))

In [ ]:
immature = crunch[['min_revenues','series.angel','series.pre_seed','series.seed','series.series_a']]

In [ ]:
immature

In [ ]:
list(enumerate(immature))

In [ ]:
for index, colname in enumerate(immature):
    if 'series' in colname:
        immature[colname].replace(np.nan, 0.0, inplace=True)

In [ ]:
immature

In [ ]:
immature['total'] = immature.loc[:, 'series.angel':'series.series_a'].sum(axis=1)

In [ ]:
immature

In [ ]:
#df['total funds']= df.iloc[:, 6:31].sum(axis =1)

In [ ]:
immature = immature.drop(columns=['series.angel', 'series.pre_seed', 'series.seed', 'series.series_a'])

In [ ]:
immature['total'] =immature['total']>0

In [ ]:
immature['total'] = immature['total'].apply(lambda x: 1 if x==True else 0)

In [ ]:
immature

In [ ]:
immature[immature['min_revenues'].isna()]['total'].value_counts()

### now let's compare with mature, companies after series a and see revenues behaviour

In [ ]:
mature = crunch[[]]

In [ ]:
test

In [ ]:
list(range(7))

In [ ]:
list(range(13, 22))

In [ ]:
[8, list(range(13, 22)), 23, 24, list(range(27,35))]

In [ ]:
mature = crunch[['min_revenues']]
for index, colname in enumerate(crunch):
    if index in [23, 24] or index in list(range(13, 22)) or index in list(range(27,35)):
        mature[colname] = crunch[colname]

In [ ]:
mature

In [ ]:
list(enumerate(crunch))

#### why the following doesn't work?

In [ ]:
#mature.loc[:, 'series.convertible_note':'series.series_unknown'].apply(lambda x: x.replace(np.nan, 0.0, inplace=True))

In [ ]:
mature.loc[:,'series.convertible_note':]

In [ ]:
list(enumerate(mature.loc[:,'series.convertible_note':]))

In [ ]:
for index, colname in enumerate(mature.loc[:,'series.convertible_note':]):
    mature[colname].replace(np.nan, 0.0, inplace=True)
    

In [ ]:
mature

In [ ]:
mature['total'] = mature.loc[:,'series.convertible_note':].sum(axis=1)

In [ ]:
mature_final = mature[['min_revenues', 'total']]

In [ ]:
mature_final['total'] =mature_final['total']>0

In [ ]:
mature_final['total'] = mature_final['total'].apply(lambda x: 1 if x ==True else 0)

In [ ]:
mature_final

### Les boites matures (series-b,c,d,e,f,gh, etc) qui n'ont pas de revenues, 2264 ont lévé

In [ ]:
mature_final[mature_final['min_revenues'].isna()]['total'].value_counts()

### les boites Matures (series-b,c,d,e,f,g,h, etc) qui ont de revenues, juste 1 a levé

In [ ]:
mature_final[mature_final['min_revenues'].notna()]['total'].value_counts()

### les boites immatures (angel, pre-seed, seed, series-a) qui n'ont pas de revenues, 2635 ont levé de fonds

In [ ]:
immature[immature['min_revenues'].isna()]['total'].value_counts()

### Les boites immatures (angel, pre-seed, seed, series-a) qui ont de revenues, aucune n'a levé

In [ ]:
immature[immature['min_revenues'].notna()]['total'].value_counts()

### On voit que les matures et immatures qui n'ont pas de revenues, correspondent à 99% qui ont levés

In [ ]:
immature.shape

In [ ]:
mature_final.shape

In [ ]:
immature.info()

In [ ]:
immature[['min_revenues', 'total_immature']] = immature

In [ ]:
immature = immature.drop(columns=['total'])

In [ ]:
immature

In [ ]:
mature_final

In [ ]:
mature_final['total_mature'] = mature_final['total']

In [ ]:
mature_final = mature_final.drop(columns=['total'])

In [ ]:
mature_final

In [ ]:
# trying = pd.merge(immature, mature_final) THIS COMMAND TOOK FOREVER, WHY?

In [ ]:
immature

In [ ]:
mature_final

In [ ]:
my_total = immature

In [ ]:
my_total['total_mature'] = mature_final['total_mature']


In [ ]:
my_total

In [ ]:
my_total['immature and mature'] = my_total[['total_immature','total_mature']].sum(axis=1)

In [ ]:
my_total

In [ ]:
my_total['immature and mature'].value_counts()

### Les immatures qui ont levés uniquement angel, pre-seed, seed, series-a sont 1542

In [ ]:
my_total[my_total['immature and mature']==1]['total_immature'].value_counts()

In [ ]:
my_total[(my_total['immature and mature']==1) & (my_total['min_revenues'].notna())]

In [ ]:
my_total[(my_total['immature and mature']==1) & (my_total['min_revenues'].isna())]

### Les immatures qui ont levés uniquement angel, pre-seed, seed, series-a, qui n'avaient pas d'info sur revenues, sont 1542

In [ ]:
my_total[(my_total['immature and mature']==1) & (my_total['min_revenues'].isna())]['total_immature'].value_counts()

### Les matures qui ont lévé, mais pas angel, ni pre-seed, ni seed, ni series-a mais le reste, sont 1172

In [ ]:
my_total[my_total['immature and mature']==1]['total_mature'].value_counts()

In [ ]:
my_total[(my_total['immature and mature']==1) & (my_total['min_revenues'].notna())]['total_mature'].value_counts()

### Les matures qui ont levés, mais pas angel ni (pre-seed, seed, series a) mais le reste, qui n'avaient pas d'infos sur de revenues, sont 1171

In [ ]:
my_total[(my_total['immature and mature']==1) & (my_total['min_revenues'].isna())]['total_mature'].value_counts()

In [ ]:
my_total[(my_total['immature and mature']==2) & (my_total['min_revenues'].isna())]

In [ ]:
my_total[(my_total['immature and mature']==2) & (my_total['min_revenues'].isna())]['total_mature'].value_counts()

In [ ]:
my_total[(my_total['immature and mature']==2) & (my_total['min_revenues'].isna())]['total_immature'].value_counts()

In [ ]:
industries_dict_complete = {
"Education": [ "Charter Schools", "Children", "Continuing Education", "Corporate Training", "E-Learning", "EdTech", "Education", "Higher Education", "Tutoring", "Universities", "Vocational Education" ],
"Business_Products_And_Services": [ "Accounting", "Advice", "Alumni", "Association", "B2B", "B2C", "Billing", "Brand Marketing", "Business Development", "Call Center", "Career Planning", "Charity", "Consulting", "Contact Management", "Coworking", "Creative Agency", "CRM", "Customer Service", "Digital Marketing", "Direct Sales", "Email Marketing", "Enterprise", "Freelance", "Information Services", "Innovation Management", "Intellectual Property", "Interior Design", "Knowledge Management", "Laundry and Dry-cleaning", "Lead Generation", "Legal", "Local", "Local Advertising", "Local Business", "Loyalty Programs", "Management Consulting", "Market Research", "Marketing", "Non Profit", "Outsourcing", "Price Comparison", "Procurement", "Product Management", "Product Research", "Professional Services", "Public Relations", "Quality Assurance", "Reservations", "Sales", "Self-Storage", "Service Industry", "Small and Medium Businesses", "Social Entrepreneurship", "Social Impact", "Social Media Management", "Social Media Marketing", "Subscription Service", "Supply Chain Management", "Ticketing", "Trade Shows", "Transaction Processing", "Translation Service", "Virtual Workforce", "Warehousing", "Wholesale" ],
"software": ["A/B Testing", "Analytics", "Application Performance Management", "Archiving Service", "Big Data", "Business Information Systems", "Business Intelligence", "CivicTech", "Cloud Computing", "Cloud Data Services", "Cloud Infrastructure", "Cloud Management", "Cloud Security", "Cloud Storage", "Collaboration", "Consumer Software", "Data Center", "Data Center Automation", "Data Integration", "Data Mining", "Data Storage", "Data Visualization", "Database", "Desktop Apps", "Developer APIs", "Developer Platform", "Developer Tools", "Document Management", "Domain Registrar", "E-Commerce Platforms", "E-Signature", "Electronic Design Automation (EDA)", "Email", "Embedded Software", "Embedded Systems", "Enterprise Applications", "Enterprise Resource Planning (ERP)", "Enterprise Software", "File Sharing", "Graphic Design", "IaaS", "Information Technology", "IT Management", "Legal Tech", "Linux", "Management Information Systems", "Marketing Automation", "Meeting Software", "MMO Games", "Online Games", "Open Source", "Operating Systems", "PaaS", "PC Games", "Photo Editing", "Presentation Software", "Productivity Tools", "Project Management", "Reading Apps", "SaaS", "Sales Automation", "Scheduling", "Serious Games", "Software", "Software Engineering", "Usability Testing", "UX Design", "Video Editing", "Virtualization", "Web Design", "Web Development", "Web Hosting"],
"media_traditional": ["Ad Exchange", "Ad Network", "Ad Retargeting", "Ad Server", "Ad Targeting", "Advertising", "Advertising Platforms", "Animation", "Art", "Blogging Platforms", "Broadcasting", "Digital Entertainment", "Digital Media", "EBooks", "Film", "Film Production", "Independent Music", "Internet Radio", "Journalism", "Media and Entertainment", "Music", "Music Label", "Music Streaming", "News", "Outdoor Advertising", "Podcast", "Politics", "Publishing", "TV", "TV Production"],
"agriculture": ["Agriculture", "AgTech", "Animal Feed", "Aquaculture", "Cannabis", "Farmers Market", "Farming", "Forestry", "Fruit", "Hydroponics", "Organic"],
"transportation": ["Air Transportation", "Automotive", "Autonomous Vehicles", "Car Sharing", "Delivery", "Delivery Service", "Electric Vehicle", "Fleet Management", "Freight Service", "Last Mile Transportation", "Logistics", "Marine Transportation", "Parking", "Public Transportation", "Railroad", "Recreational Vehicles", "Ride Sharing", "Same Day Delivery", "Shipping", "Space Travel", "Taxi Service", "Transportation", "Water Transportation"],
"Healthcare": ['Alternative Medicine', 'Bioinformatics', 'Biopharma', 'Biotechnology', 'Child Care', 'Clinical Trials', 'Cosmetic Surgery', 'Dental', 'Elder Care', 'Elderly', 'Electronic Health Record (EHR)', 'Emergency Medicine', 'Funerals', 'Genetics', 'Health', 'Health Care', 'Health Diagnostics', 'Home Health Care', 'Hospital', 'Life Science', 'Medical', 'Medical Device', 'Neuroscience', 'Nursing and Residential Care', 'Personal Health', 'Pharmaceutical', 'Quantified Self', 'Rehabilitation', 'Therapeutics', 'Veterinary', 'Wellness'],
"Mobile_And_Telecommunications": ['Android', 'App Marketing', 'Apps', 'Augmented Reality', 'Communications Infrastructure', 'GPS', 'iOS', 'Location Based Services', 'Messaging', 'Mobile', 'Mobile Advertising', 'Mobile Apps', 'Mobile Devices', 'NFC', 'QR Codes', 'Satellite Communication', 'SMS', 'Telecommunications', 'Unified Communications', 'Video Conferencing', 'VoIP', 'Wireless'],
"Financial": ["Angel Investment", "Asset Management", "Auto Insurance", "Banking", "Bitcoin", "Blockchain", "Consumer Lending", "Credit", "Credit Cards", "Crowdfunding", "Cryptocurrency", "Debit Cards", "Debt Collections", "Emerging Markets", "Ethereum", "Finance", "Financial Exchanges", "Financial Services", "FinTech", "Funding Platform", "Health Insurance", "Hedge Funds", "Impact Investing", "Insurance", "InsurTech", "Leasing", "Lending", "Life Insurance", "Mobile Payments", "Payments", "Personal Finance", "Prediction Markets", "Property Insurance", "Real Estate Investment", "Retirement", "Risk Management", "Trading Platform", "Venture Capital", "Wealth Management"],
"artificial_intelligence": ["Artificial Intelligence", "Computer Vision", "Facial Recognition", "Gamification", "Human Computer Interaction", "Image Recognition", "Intelligent Systems", "Language Learning", "Machine Learning", "MOOC", "Motion Capture", "Natural Language Processing", "Personalization", "Predictive Analytics", "Quantum Computing", "Real Time", "Robotics", "Semantic Search", "Semantic Web", "Simulation", "Smart Building", "Smart Cities", "Smart Home", "Speech Recognition", "Text Analytics", "Virtual Assistant", "Virtual Reality", "Visual Search"],
"Retail_NonInternetMobile": ['Auctions', 'Beauty', 'Classifieds', 'Collaborative Consumption', 'Consumer', 'Consumer Goods', 'Cosmetics', 'Coupons', 'DIY', 'E-Commerce', 'Eyewear', 'Fashion', "Fast-Moving Consumer Goods", 'Flowers', 'Gift', 'Gift Card', 'Jewelry', 'Lingerie', 'Local Shopping', "Men's", 'Musical Instruments', 'Rental', 'Retail', 'Shoes', 'Shopping', 'Social Shopping', 'Sporting Goods', 'Toys', "Women's"],
"Electronics": ['Audio', 'Consumer Electronics', 'Digital Signage', 'Electronics', 'Laser', 'Lighting', 'Optical Communication', 'RFID', 'Semiconductor', 'Sensor', 'Wearables'],
"Food_And_Beverages": ['Baby', 'Bakery', 'Catering', 'Coffee', 'Confectionery', 'Cooking', 'Craft Beer', 'Dietary Supplements', 'Food and Beverage', 'Food Delivery', 'Food Processing', 'Grocery', 'Nutraceutical', 'Nutrition', 'Organic Food', 'Pet', 'Recipes', 'Restaurants', 'Seafood', 'Snack Food', 'Tea', 'Wine And Spirits', 'Winery'],
"energy_and_utilities": ["Battery", "Biomass Energy", "Clean Energy", "CleanTech", "Electrical Distribution", "Energy", "Energy Efficiency", "Energy Management", "Energy Storage", "Fossil Fuels", "Fuel", "Fuel Cell", "Infrastructure", "Natural Resources", "Nuclear", "Oil and Gas", "Power Grid", "Renewable Energy", "Solar", "Water", "Wind Energy"],
"risk_and_security": ['Biometrics', 'Compliance', 'Cyber Security', 'Fraud Detection', 'Homeland Security', 'Identity Management', 'Intrusion Detection', 'Law Enforcement', 'Military', 'Network Security', 'Physical Security', 'Privacy', 'Public Safety', 'Reputation', 'Security'],
"Computer_Hardware_And_Services": ['CAD', 'Computer', 'Hardware', 'IT Infrastructure', 'Network Hardware', 'Point of Sale', 'Private Cloud', 'Retail Technology', 'Technical Support'],
"real_estate": ["Commercial Real Estate", "Property Development", "Property Management", "Real Estate", "Rental Property", "Residential", "Timeshare", "Travel Accommodations", "Vacation Rental"],
"internet": ["Communities", "Consumer Applications", "Consumer Reviews", "Content", "Content Creators", "Content Delivery Network", "Content Discovery", "Content Marketing", "Crowdsourcing", "ICT", "Internet", "Internet of Things", "ISP", "Mapping Services", "Marketplace", "Navigation", "Online Auctions", "Online Portals", "Peer to Peer", "Photo Sharing", "Private Social Networking", "Product Search", "Q&A", "Search Engine", "SEM", "SEO", "Sharing Economy", "Social", "Social Bookmarking", "Social Media", "Social Media Advertising", "Social Network", "Social News", "Video", "Video Advertising", "Video Chat", "Video on Demand", "Video Streaming", "Virtual World", "Web Apps", "Web Browsers"],
"hr_people_needs": ['Employee Benefits', 'Employment', 'Family', 'Human Resources', 'Parenting', 'Personal Branding', 'Personal Development', 'Professional Networking', 'Recruiting', 'Skill Assessment', 'Social Recruiting', 'Staffing Agency', 'Training'],
"environmental_services_and_equipment": ['Environmental Consulting', 'Environmental Engineering', 'Green Building', 'Green Consumer Goods', 'GreenTech', 'Janitorial Service', 'Landscaping', 'Marine Technology', 'Packaging Services', 'Parks', 'Pollution Control', 'Printing', 'Recycling', 'Sustainability', 'Waste Management', 'Water Purification']
}


In [ ]:
industries_dict_complete.keys()

In [ ]:
industries_dict_complete.values()

In [ ]:
crunch['industries_list']

In [ ]:
for industry in crunch['industries_list'][0]:
    print(industry)
    if industry in industries_dict_complete.values():
        print(industry)

In [ ]:
index, *value = crunch[['industries_list']][:10].iterrows()

In [ ]:
industria = crunch[['industries_list']][:2].iterrows()

In [ ]:
list(industria)

In [ ]:
#industria.shape

In [ ]:
index

In [ ]:
value

In [ ]:
list(crunch[['industries_list']][:10].iterrows())

In [ ]:
crunch[['industries_list']][:10].iterrows()

In [ ]:
for bulk in industries_dict_complete.values():
    for index, industry in crunch[['industries_list']][:10].iterrows():
        if bulk in industry['industries_list']:
            print(industry)

In [ ]:
test_df = crunch[['industries_list']][:10]

In [ ]:
test_df

In [ ]:
my_industries = ['Finance', 'Financial Services', 'FinTech']

In [ ]:
cleaning_ind_complete = lambda row: set( [k for k, v in industries_dict_complete.items() if set(row).intersection(set(v)) ])

In [ ]:
#another_test = cleaning_ind_complete(test_df['industries_list'])

### CLEANING THE NEW INDUSTRY DICT COMPLETE THE RESULT, THE WHOLE PROCESS IS BEFORE

In [ ]:
another_test = test_df['industries_list'].apply(cleaning_ind_complete)

In [ ]:
another_test

In [ ]:
test_df['industries_list']

In [ ]:
#clean_industries_list = lambda row: set([k for k, v in industries_dict.items() if set(row).intersection(set(v))])

In [ ]:
crunch[['industries_list']].apply()

In [ ]:
for index, row in crunch[['industries_list']][:10].iterrows():
    for industry in my_industries:
        if industry in row['industries_list']:
            print(f'the index is {index}')
            print(industry)

In [ ]:
crunch

In [ ]:
df

1. quel modèles scikitlearn classifier
2. - country_code = onehotencoder
- investors list onehotencoder
- industries_cleaned = onehotencoder
- employee_range onehotencoder


In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(df['country_code'], df['employee_range'])

In [ ]:
for country in df['country_code'].unique():
    print(country)
    plt.scatter(df['country_code'].str.contains(country), df['employee_range'])
    plt.xlabel(f'the country is {country}')
    plt.ylabel('Number of employees')
    plt.show()

In [ ]:
plt.scatter(df['min_revenues'], df['employee_range'])

In [ ]:
plt.scatter(df['min_revenues'], df['traffic.monthly'])

In [ ]:
#df.duplicated()
df.loc[df.astype(str).duplicated().index]

In [ ]:
df.astype(str).duplicated()

In [ ]:
df.astype(str).duplicated().sum()

In [ ]:
#df = df.drop_duplicates()
df = df.astype(str).drop_duplicates()

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df.isnull().sum().sort_values(ascending=False) 

In [ ]:
df.isna().sum().sort_values(ascending=False) 

In [ ]:
df

In [ ]:
df.isnull().sort_values(by='traffic.global_rank',ascending=False) 

In [ ]:
df.notna().sort_values(by='traffic.global_rank',ascending=False) 

In [ ]:
df.sort_values(by='traffic.global_rank',ascending=False) 

In [ ]:
type(df['traffic.monthly'][243266])

In [ ]:
for index, colname in enumerate(df):
    df[colname].replace('nan', np.nan, inplace=True)

In [ ]:
df['traffic.monthly'].astype(str)

In [ ]:
df['traffic.monthly'].astype(str)=='nan'

In [ ]:
type(df['traffic.monthly'][311409])

In [ ]:
#df['traffic.monthly'].isna().sum().sort_values()

### trafic monthly has 85011 nan values

In [ ]:
df['traffic.monthly'].isna().value_counts()

In [ ]:
df['traffic.monthly'].isna()

In [ ]:
df['traffic.global_rank'].isna().value_counts()

In [ ]:
for index, colname in enumerate(df):
    print(f'this is the column {colname} and these are the values: \n{df[colname].isna().value_counts()}')
    print()

In [ ]:
136256-132839

In [ ]:
df[df['investors_list'].astype(str)=='[]']['investors_list'].value_counts()

In [ ]:
#df[df['total funds']>0]['total funds'].value_counts()

In [ ]:
df['total funds'].astype(str)

In [ ]:
df['total funds'].astype(str)[0]

In [ ]:
type(df['total funds'].astype(str)[0])

In [ ]:
df.info()

In [ ]:
#df[['min_revenues']].boxplot()

In [ ]:
df['min_revenues'].astype(str)

In [ ]:
df['min_revenues'].astype(str).value_counts()

In [ ]:
df['min_revenues'].isna().value_counts()

In [ ]:
type(df['min_revenues'][311411])

In [ ]:
df['min_revenues'][311412] == np.nan

In [ ]:
df['min_revenues'][311412] == 'nan'

In [ ]:
df['min_revenues'].replace('nan')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df

In [ ]:
crunch.columns

In [ ]:
df['total funds'].replace('1', int(1), inplace=True)

In [ ]:
df['total funds'].replace('0', int(0), inplace=True)

In [ ]:
df['total funds'].astype(str).value_counts()

In [ ]:
type(df['total funds'][4])

In [ ]:
df['min_revenues']

In [ ]:
df

In [ ]:
df['traffic.monthly'].notna().value_counts()

In [ ]:
df[df['traffic.monthly'].notna()]['total funds'].value_counts()

In [ ]:
df[df['traffic.monthly'].isna()]['total funds'].value_counts()

In [ ]:
df[df['investors_list']=='[]']['total funds'].value_counts()

### Nous avouns industrie, head count, country

### est-ce que pas d'investisseurs ou c'est parce qu'on n'a pas les données

In [ ]:
df[df['total funds']==1]['investors_list'].isna().value_counts()

In [ ]:
#df[['traffic.global_rank']].boxplot();

In [ ]:
df['traffic.global_rank'].apply(type).eq(str).value_counts()

In [ ]:
#df['traffic.global_rank'] = df[df['traffic.global_rank'].apply(type).eq(str)]['traffic.global_rank'].apply(lambda x: float(x))
# NOT DO THIS! it equaled the good values, but added nan's everywhere because it was a boolean mask

In [ ]:
df[df['traffic.global_rank'].apply(type).eq(str)]['traffic.global_rank']

In [ ]:
df['traffic.global_rank']